In [ ]:
pip install gradio PyPDF2 langchain-google-genai langchain-core langgraph


In [ ]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
pip install langchain[google]

In [ ]:
%pip install --upgrade --quiet tiktoken langchain langgraph beautifulsoup4 langchain langchain-google-genai langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.8 MB/s eta 0:00:00


In [ ]:
import os
import asyncio
from typing import List
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from PyPDF2 import PdfReader
import gradio as gr

# Configure Google Gemini 1.5 Flash LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Utility to extract text from a PDF
def extract_text_from_pdf(pdf_path: str) -> List[str]:
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text.split("\n\n")  # Split into chunks (paragraphs)

# Iterative Refinement Components
summarize_prompt = ChatPromptTemplate([("human", "Write a concise summary of the following: {context}")])
initial_summary_chain = summarize_prompt | llm | StrOutputParser()

refine_template = """
Produce a final summary.

Existing summary up to this point:
{existing_answer}

New context:
------------
{context}
------------

Given the new context, refine the original summary.
"""
refine_prompt = ChatPromptTemplate([("human", refine_template)])
refine_summary_chain = refine_prompt | llm | StrOutputParser()

# Iterative Refinement Function
async def iterative_refinement(chunks: List[str]) -> str:
    summary = await initial_summary_chain.ainvoke(chunks[0], {})
    for chunk in chunks[1:]:
        summary = await refine_summary_chain.ainvoke(
            {"existing_answer": summary, "context": chunk}, {}
        )
    return summary

# Map-Reduce Components
map_prompt = ChatPromptTemplate([("human", "Write a concise summary of the following:\n\n{context}")])
map_chain = map_prompt | llm | StrOutputParser()

reduce_template = """
The following is a set of summaries:
{docs}
Take these and distill them into a final, consolidated summary of the main themes.
"""
reduce_prompt = ChatPromptTemplate([("human", reduce_template)])
reduce_chain = reduce_prompt | llm | StrOutputParser()

# Map-Reduce Function
async def map_reduce_summary(chunks: List[str]) -> str:
    summaries = []

    # Step 1: Generate summaries for each chunk (Map Step)
    for chunk in chunks:
        summary = await map_chain.ainvoke(chunk, {})
        summaries.append(summary)

    # Step 2: Consolidate the summaries into a final summary (Reduce Step)
    consolidated_summaries = "\n".join(summaries)
    final_summary = await reduce_chain.ainvoke({"docs": consolidated_summaries}, {})
    return final_summary

# Async Wrapper for PDF Processing
async def process_pdf_async(pdf_file, method):
    if not pdf_file:
        return "Please upload a valid PDF file."

    # Extract text from uploaded PDF
    pdf_path = pdf_file.name
    chunks = extract_text_from_pdf(pdf_path)

    # Choose Summarization Method
    if method == "Iterative Refinement":
        summary = await iterative_refinement(chunks)
    elif method == "Map-Reduce":
        summary = await map_reduce_summary(chunks)
    else:
        summary = "Invalid method selected."
    return summary

# Gradio Interface
iface = gr.Interface(
    fn=process_pdf_async,  # Use the async function directly here
    inputs=[
        gr.File(label="Upload PDF"),
        gr.Radio(["Iterative Refinement", "Map-Reduce"], label="Choose Summarization Method"),
    ],
    outputs="text",
    title="PDF Summarization",
    description="Upload a PDF and select a summarization method to generate a summary.",
)

# Launch Gradio App
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8a74562948873635f1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
